# ETL - Ventas

## 1. Extracción.

### 1.1. importamos los DataFrame de Ventas por año y los unimos.

In [2]:
import pandas as pd
import os

# Carpeta donde están los archivos
data_path = "data/raw/"
# Rango de años
anios = range(2018, 2025)

dfs = []

for anio in anios:
    parquet_file = os.path.join(data_path, f"fact_ventas_{anio}.parquet")
    csv_file = os.path.join(data_path, f"fact_ventas_{anio}.csv")
    
    try:
        if os.path.exists(parquet_file):
            df = pd.read_parquet(parquet_file)
            print(f"✔️ Archivo parquet cargado: {parquet_file}")
        elif os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            print(f"✔️ Archivo CSV cargado: {csv_file}")
        else:
            print(f"⚠️ No se encontró archivo para el año {anio}")
            continue
        
        # Agregamos columna de trazabilidad
        df["anio_origen"] = anio
        # Guardamos en lista
        dfs.append(df)
    
    except Exception as e:
        print(f"❌ Fallo al cargar {anio}: {e}")

# Concatenamos todos los dataframes
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    print(f"[INFO] Dataset final armado con {len(df_final)} registros de {len(dfs)} años")
else:
    df_final = pd.DataFrame()
    print("[INFO] No se cargaron datos")

df = df_final

✔️ Archivo CSV cargado: data/raw/fact_ventas_2018.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2019.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2020.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2021.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2022.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2023.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2024.csv
[INFO] Dataset final armado con 839071 registros de 7 años


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839071 entries, 0 to 839070
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   transaction_id               839071 non-null  int64  
 1   fecha                        839071 non-null  object 
 2   año                          839071 non-null  int64  
 3   mes                          839071 non-null  int64  
 4   cliente_id                   839071 non-null  int64  
 5   producto_id                  839071 non-null  int64  
 6   canal_id                     839071 non-null  int64  
 7   sucursal_id                  467750 non-null  float64
 8   cantidad                     839071 non-null  int64  
 9   precio_unitario_ars_nominal  839071 non-null  float64
 10  monto_venta_ars_nominal      839071 non-null  float64
 11  monto_venta_ars_real_2018    839071 non-null  float64
 12  es_evento                    839071 non-null  int64  
 13 

In [4]:
df.head()

,transaction_id,fecha,año,mes,cliente_id,producto_id,canal_id,sucursal_id,cantidad,precio_unitario_ars_nominal,monto_venta_ars_nominal,monto_venta_ars_real_2018,es_evento,anio_origen
0,1,2018-01-01,2018,1,9970,5,2,5.0,1,1460.35,1460.35,1460.35,0,2018
1,2,2018-01-01,2018,1,11865,17,2,19.0,1,19129.42,19129.42,19129.42,0,2018
2,3,2018-01-01,2018,1,10232,18,1,NaN,2,17325.83,34651.66,34651.66,0,2018
3,4,2018-01-01,2018,1,8257,9,2,25.0,1,6015.50,6015.50,6015.50,0,2018
4,5,2018-01-01,2018,1,8053,1,1,NaN,2,30871.57,61743.15,61743.15,0,2018


## 2. Transformación

### 2.1. Busqueda de Duplicados

In [5]:
# Filtrar duplicados (todas las columnas)
df_duplicados = df[df.duplicated()] # Busca filas duplicadas o sea dos filas exactamente iguales
print("Registros Duplicados:", df_duplicados)
# Filtrar duplicados (columnas críticas)
df_duplicados_id = df[df.duplicated(subset=["transaction_id"], keep=False)]
print("transaction_id (duplicados):\n", df_duplicados_id)


Registros Duplicados: Empty DataFrame
Columns: [transaction_id, fecha, año, mes, cliente_id, producto_id, canal_id, sucursal_id, cantidad, precio_unitario_ars_nominal, monto_venta_ars_nominal, monto_venta_ars_real_2018, es_evento, anio_origen]
Index: []
transaction_id (duplicados):
 Empty DataFrame
Columns: [transaction_id, fecha, año, mes, cliente_id, producto_id, canal_id, sucursal_id, cantidad, precio_unitario_ars_nominal, monto_venta_ars_nominal, monto_venta_ars_real_2018, es_evento, anio_origen]
Index: []


### 2.2. Busqueda Valores Null (NaN)

In [6]:
print(df.isnull().sum()) # Muestra valores nulos (valor cero no es nulo)

transaction_id                      0
fecha                               0
año                                 0
mes                                 0
cliente_id                          0
producto_id                         0
canal_id                            0
sucursal_id                    371321
cantidad                            0
precio_unitario_ars_nominal         0
monto_venta_ars_nominal             0
monto_venta_ars_real_2018           0
es_evento                           0
anio_origen                         0
dtype: int64


Esos NaN en sucursal_id podrian ser ventas online, asi que lo verificaremos

In [7]:
ventas_online = df[df["sucursal_id"].isna() & (df["canal_id"] == 1)]
print(ventas_online.head())

# Contar valores nulos en la columna 'sucursal_id'
nulos_sucursal = df["sucursal_id"].isna().sum()
print("Nulos en sucursal_id: ", nulos_sucursal)
print("Ventas Online:     -> ", ventas_online.shape[0])


    transaction_id       fecha   año  mes  cliente_id  producto_id  canal_id  \
2                3  2018-01-01  2018    1       10232           18         1   
4                5  2018-01-01  2018    1        8053            1         1   
6                7  2018-01-01  2018    1        1814            9         1   
9               10  2018-01-01  2018    1        7154           13         1   
16              17  2018-01-01  2018    1       12326           11         1   

    sucursal_id  cantidad  precio_unitario_ars_nominal  \
2           NaN         2                     17325.83   
4           NaN         2                     30871.57   
6           NaN         2                      6109.52   
9           NaN         1                     14585.79   
16          NaN         2                     20151.13   

    monto_venta_ars_nominal  monto_venta_ars_real_2018  es_evento  anio_origen  
2                  34651.66                   34651.66          0         2018  
4       

In [8]:
# Reemplazar NaN en sucursal_id por 0 que haremos que sea para ventas online
df.loc[df["sucursal_id"].isna(), "sucursal_id"] = 0

print("Registros Modificados: ", (df["sucursal_id"] == 0).sum())

Registros Modificados:  371321


In [9]:
sucursal_repetida = 26  # reemplazá por el valor real

df_check = df[df["sucursal_id"] == sucursal_repetida]

if df_check.empty:
    print(f"✔️ La sucursal '{sucursal_repetida}' NO aparece en fact_ventas")
else:
    print(f"⚠️ La sucursal '{sucursal_repetida}' aparece {len(df_check)} veces en fact_ventas")
    print(df_check.head())

✔️ La sucursal '26' NO aparece en fact_ventas


## 3. Carga

### 3.1. Guardamos el DF transformado y limpio.

In [11]:
import os

filename = "data/processed/fact_ventas.parquet"

try:
    df.to_parquet(filename, engine="fastparquet")
    if os.path.exists(filename):
        print(f"✔️ Dataset guardado con éxito en: {filename}")
    else:
        print("⚠️ No se encontró el archivo después de guardar.")
except Exception as e:
    print(f"❌ Error al guardar el dataset: {e}")

✔️ Dataset guardado con éxito en: data/processed/fact_ventas.parquet


### 3.2. Generamos la Tabla Calendar

In [ ]:
import pandas as pd

fechas = pd.date_range(
    start=df["fecha"].min(),
    end=df["fecha"].max(),
    freq="D"
)

dias = {
    "Monday":"Lunes","Tuesday":"Martes","Wednesday":"Miércoles",
    "Thursday":"Jueves","Friday":"Viernes","Saturday":"Sábado","Sunday":"Domingo"
}
meses = {
    "January":"Enero","February":"Febrero","March":"Marzo","April":"Abril",
    "May":"Mayo","June":"Junio","July":"Julio","August":"Agosto",
    "September":"Septiembre","October":"Octubre","November":"Noviembre","December":"Diciembre"
}

calendario = pd.DataFrame({"fecha": fechas})
calendario["año"] = calendario["fecha"].dt.year
calendario["mes"] = calendario["fecha"].dt.month
calendario["nombre_mes"] = calendario["fecha"].dt.strftime("%B").map(meses)
calendario["trimestre"] = "T" + calendario["fecha"].dt.quarter.astype(str)
calendario["semana_año"] = calendario["fecha"].dt.isocalendar().week
calendario["día"] = calendario["fecha"].dt.day
calendario["nombre_día"] = calendario["fecha"].dt.strftime("%A").map(dias)


In [ ]:
calendario.head()

### 3.3. Guardamos el DF Calendario.

In [ ]:
import os

filename = "data/processed/tecnoStore/dim_calendario.parquet"

try:
    calendario.to_parquet(filename, index=False)
    if os.path.exists(filename):
        print(f"✔️ Dataset guardado con éxito en: {filename}")
    else:
        print("⚠️ No se encontró el archivo después de guardar.")
except Exception as e:
    print(f"❌ Error al guardar el dataset: {e}")